# Модель BERT и GPT

#### Задание

1. Взять датасет

https://huggingface.co/datasets/merionum/ru_paraphraser

решить задачу парафраза

2. (дополнительно необязательная задача) на выбор взять

https://huggingface.co/datasets/sberquad

https://huggingface.co/datasets/blinoff/medical_qa_ru_data

натренировать любую модель для вопросно ответной системы

### Парафраз

In [33]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [35]:
# Загрузка датасета

from datasets import load_dataset
dataset = load_dataset("merionum/ru_paraphraser")
dataset

Using custom data configuration merionum--ru_paraphraser-1a7592429d7be082
Reusing dataset json (/root/.cache/huggingface/datasets/merionum___json/merionum--ru_paraphraser-1a7592429d7be082/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 7227
    })
    test: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 1924
    })
})

In [36]:
dataset['train'][1]

{'class': '0',
 'id': '2',
 'id_1': '202',
 'id_2': '8158',
 'text_1': 'Право полицейских на проникновение в жилище решили ограничить.',
 'text_2': 'Правила внесудебного проникновения полицейских в жилище уточнят.'}

In [37]:
!pip install transformers[sentencepiece] 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [39]:
# Загрузка предобученных моделей и пайплайна
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rut5-base-paraphraser")
model = AutoModelForSeq2SeqLM.from_pretrained("cointegrated/rut5-base-paraphraser")

/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


In [40]:
text = 'В Минобороны РФ заявили об уничтожении более 1 тыс. снарядов к гаубицам M777'

In [47]:
x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
x

{'input_ids': tensor([[  635,  5584, 25685,   433,   259,  4934,  3562,  1533,  1091,   456,
         17438,  3695,   259,  2611,   333,  9545,   260,   388, 18737,   685,
           778,   259,  1018, 25047, 15092,   352,   487,  2320,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1]])}

In [57]:
for inx2tok in x['input_ids'][0]:
  print(tokenizer.decode(inx2tok))

В
Мин
оборон
ы

РФ
заяв
или
об
у
ничтож
ении

более
1
тыс
.
с
наряд
ов
к

га
уби
цам
M
7
77
</s>


In [42]:
# Функция парафраза

def paraphrase(text, beams=15, grams=2, do_sample=False):
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    print(x, x.input_ids.shape[1])
    max_size = int(x.input_ids.shape[1] * 1.5 + 10)
    out = model.generate(**x, encoder_no_repeat_ngram_size=grams, num_beams=beams, max_length=max_size, do_sample=do_sample)
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [43]:
import random

In [44]:
# Примеры работы парафраза grams = 2

for i in range(5):
    k = i#random.randint(1,100)
    print('Исходный текст: ', dataset["train"][k]['text_1'])
    print('Парафраз: ', paraphrase(dataset["train"][k]['text_1']))
    print('------------------------------------------------------------')

Исходный текст:  Полицейским разрешат стрелять на поражение по гражданам с травматикой.
{'input_ids': tensor([[ 1152, 19044,  6863, 11087,  1184, 15972,  2552,   310,   425, 19141,
           425,  5755,  1164,   388,   259, 28900,   279,  2245,   260,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])} 20
Парафраз:  Полиция разрешит стрельбу на нападение на граждан, которые имеют травмы, и полицейские будут разрешать стрельбу на нападение на людей с
------------------------------------------------------------
Исходный текст:  Право полицейских на проникновение в жилище решили ограничить.
{'input_ids': tensor([[20136,   425, 19044,  3351,   310, 18428,  6054,  1733,   315,  4398,
          9174,  3540,  1533, 11882,  1158,   260,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])} 17
Парафраз:  Ограничить право полиции проникнуть в квартиру было решено в рамках проекта «Ограничить право полиции на въ

In [58]:
# Примеры работы парафраза grams = 4

for i in range(5):
    k = i#random.randint(1,100)
    print('Исходный текст: ', dataset["train"][k]['text_1'])
    print('Парафраз: ', paraphrase(dataset["train"][k]['text_1'], grams=4))
    print('------------------------------------------------------------')

Исходный текст:  Полицейским разрешат стрелять на поражение по гражданам с травматикой.
{'input_ids': tensor([[ 1152, 19044,  6863, 11087,  1184, 15972,  2552,   310,   425, 19141,
           425,  5755,  1164,   388,   259, 28900,   279,  2245,   260,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])} 20
Парафраз:  Полицейские могут стрелять на нападение на граждан с травматизмом.
------------------------------------------------------------
Исходный текст:  Право полицейских на проникновение в жилище решили ограничить.
{'input_ids': tensor([[20136,   425, 19044,  3351,   310, 18428,  6054,  1733,   315,  4398,
          9174,  3540,  1533, 11882,  1158,   260,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])} 17
Парафраз:  Решили ограничить право полицейских проникнуть в жилище.
------------------------------------------------------------
Исходный текст:  Президент Египта ввел чрезвычайное положе

In [59]:
# Примеры работы парафраза grams = 6

for i in range(5):
    k = i#random.randint(1,100)
    print('Исходный текст: ', dataset["train"][k]['text_1'])
    print('Парафраз: ', paraphrase(dataset["train"][k]['text_1'], grams=6))
    print('------------------------------------------------------------')

Исходный текст:  Полицейским разрешат стрелять на поражение по гражданам с травматикой.
{'input_ids': tensor([[ 1152, 19044,  6863, 11087,  1184, 15972,  2552,   310,   425, 19141,
           425,  5755,  1164,   388,   259, 28900,   279,  2245,   260,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])} 20
Парафраз:  Полиция разрешит стрелять на поражение гражданам с травматизмом.
------------------------------------------------------------
Исходный текст:  Право полицейских на проникновение в жилище решили ограничить.
{'input_ids': tensor([[20136,   425, 19044,  3351,   310, 18428,  6054,  1733,   315,  4398,
          9174,  3540,  1533, 11882,  1158,   260,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])} 17
Парафраз:  Полицейские решили ограничить право проникновения в жилище.
------------------------------------------------------------
Исходный текст:  Президент Египта ввел чрезвычайное полож

In [60]:
# Примеры работы парафраза 
text = 'В Минобороны РФ заявили об уничтожении более 1 тыс. снарядов к гаубицам M777'
print('Исходный текст: ', text)
print('Парафраз: ', paraphrase(text, grams=4))


Исходный текст:  В Минобороны РФ заявили об уничтожении более 1 тыс. снарядов к гаубицам M777
{'input_ids': tensor([[  635,  5584, 25685,   433,   259,  4934,  3562,  1533,  1091,   456,
         17438,  3695,   259,  2611,   333,  9545,   260,   388, 18737,   685,
           778,   259,  1018, 25047, 15092,   352,   487,  2320,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1]])} 29
Парафраз:  Минобороны России заявило о попытке уничтожить более 1 тысячи снарядов для гаубиц М777
